# Functional Design Patterns in Python

## Immutable Data Structures

### Tuple vs List

In [1]:
num_list = [1,2,3]
num_list[0] = 4
print(num_list)

[4, 2, 3]


In [2]:
num_tuple = (1,2,3)
num_tuple[0] = 4

TypeError: 'tuple' object does not support item assignment

### Namedtuple vs Class vs Dict

In [3]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y

point = Point(1,2)
print(f"x = {point.x}, y = {point.y}")
point.x = 3
print(f"x = {point.x}, y = {point.y}")

x = 1, y = 2
x = 3, y = 2


In [4]:
point = {"x": 1, "y": 2}
print(point)
point["x"] = 3
print(point)

{'x': 1, 'y': 2}
{'x': 3, 'y': 2}


In [5]:
from collections import namedtuple

Point = namedtuple('Point', ['x', 'y'])
point = Point(1,2)
print(f"x = {point.x}, y = {point.y}")
point.x = 3

x = 1, y = 2


AttributeError: can't set attribute

Note: Using `._replace` method in namedtuple creates a new `Point` namedtuple instance without replacing the original instance.

In [6]:
point._replace(x=3)

Point(x=3, y=2)

In [7]:
point

Point(x=1, y=2)

## Structural Pattern Matching with Dataclasses (Python equivalent of Scala case classes)

In [8]:
from dataclasses import dataclass

@dataclass(frozen='true')
class InventoryItem:
  item_name: str
  unit_price: float
  quantity: int = 0

@dataclass(frozen='true')
class SaleItem:
  item_name: str
  unit_price: float
  discount: float = 5.0

def get_greeting(request=None):
  match request:
    case InventoryItem(item_name, unit_price, quantity):
      return f"Inventory: {quantity} {item_name} at unit cost ${unit_price}"
    case SaleItem(item_name, unit_price, discount):
      return f"Selling {item_name} at ${unit_price * (100.0 - discount) / 100.0:.2f}"
    case _:
      return "Welcome to Uniqlo!"

In [9]:
get_greeting()

'Welcome to Uniqlo!'

In [10]:
get_greeting(InventoryItem("pants", 20, 300))

'Inventory: 300 pants at unit cost $20'

In [11]:
get_greeting(SaleItem("pants", 50, 15))

'Selling pants at $42.50'

## Type Hints and Type Checking

Install mypy on Google colab (source: https://stackoverflow.com/questions/63142182/to-what-extent-does-google-colab-support-python-typing)

In [12]:
# Simple mypy cell magic for Colab
!pip install mypy
from IPython.core.magic import register_cell_magic
from IPython import get_ipython
from mypy import api

@register_cell_magic
def mypy(line, cell):
  for output in api.run(['-c', '\n' + cell] + line.split()):
    if output and not output.startswith('Success'):
      raise TypeError(output)
  get_ipython().run_cell(cell)

In [13]:
%%mypy

def compute_margin(revenue: float, unit_cost: float, quantity: int) -> float:
  '''
  Compute gross margin of products sold
  '''
  return revenue - (unit_cost * quantity)

compute_margin("100.0", 5.0, 10)

TypeError: <string>:9: error: Argument 1 to "compute_margin" has incompatible type "str"; expected "float"
Found 1 error in 1 file (checked 1 source file)


## Functional Core, Imperative Shell

Let's generate some fake data first:

In [14]:
!pip install faker

In [15]:
import os
from faker import Faker
import pandas as pd

In [16]:
if os.path.isfile('data/sales.csv'):
    print("Input file exists")
else:
    fake = Faker()
    sales = {fake.company(): [fake.pyfloat(right_digits=2, min_value=0, max_value=1000), fake.pyint(min_value=0, max_value=100)] for _ in range(100)}
    price_df = pd.DataFrame.from_dict(sales, orient="index", columns=["price", "qty"])
    output_df = price_df.reset_index().rename(columns={"index": "company"})
    output_df.to_csv("data/sales.csv")

Input file exists


In [17]:
def compute_revenue(sales_item: list) -> float:
  price, qty = sales_item[1], sales_item[2]
  return [sales_item[0], price, qty, price * qty]

def main():
  # import modules
  import pandas as pd
  import numpy as np

  # I/O layer to read data into program
  data = pd.read_csv('data/sales.csv', index_col=0)
  data_cols = data.columns.tolist()

  # Functional layer for computation logic
  data_array = data.to_numpy()
  revenue_array = list(map(compute_revenue, data_array))
  revenue_cols = data_cols + ['revenue']
  revenue_df = pd.DataFrame(revenue_array, columns=revenue_cols)

  # I/O layer to write data outside of program
  revenue_df.to_csv('data/revenue.csv', index=False)

In [18]:
main()

In [19]:
pd.read_csv('data/revenue.csv', index_col=0)

,price,qty,revenue
company,,,
"Carey, Duran and Hernandez",631.16,41,25877.56
Wade-Mercer,425.98,62,26410.76
"Roberts, Walker and Mills",434.98,18,7829.64
Walker and Sons,870.54,98,85312.92
"Acevedo, Casey and Zuniga",757.31,96,72701.76
...,...,...,...
Duncan-Salas,875.72,26,22768.72
Moore-Lee,196.47,21,4125.87
Davenport and Sons,181.84,77,14001.68
